#LSH on Amazon reviews
This project is about apply a dimentionality reduction, using Local Sensitive Hashing on Amazon reviews.
Give the size of data we are pushed to spply a sort of reduction to evaluate is there are similar (or cluster os documents) so staring from the enteir text I've performe a preprocessing and a Tf-IDf to reppresent them as vectors in a multidimentional space.
To reach a good signature matrix I've transformed the Tf-IDf matrid using the Hypoerplanes method with size of 900 that could lead to a defenely better usage of this vector.
Then with hash functions for each ban we produce a bucket key, that will open a bucket in which we will find all the similar (due to the approximation) documents.

## Data Loading & Import 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

 99% 239M/242M [00:11<00:00, 23.1MB/s]
100% 242M/242M [00:11<00:00, 21.4MB/s]


In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
from zipfile import ZipFile
with ZipFile('amazon-fine-food-reviews.zip', 'r') as zipObj:
   zipObj.extractall()

In [ ]:
import pandas as pd
import collections
from operator import xor
import collections
from itertools import combinations
from scipy.stats import ttest_ind
from scipy.stats import pearsonr 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
stop_words=set(stopwords.words('english'))
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
import re
import numpy as np
contractions_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}
from itertools import combinations
from random import shuffle
import collections
import json
import spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
def multiple_replace(dict, text):
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import pickle
from numpy import random
from tqdm.notebook import tqdm
import timeit
from tqdm.notebook import tqdm
import timeit
from tqdm.contrib import itertools

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
raw_pd = pd.read_csv('Reviews.csv', sep = ',')
Text=raw_pd["Text"]

## Assignment 1.
The goal of the first assignment is to use LSH for cosine similarity. In this case, you are requested to write a Colab notebook that applies LSH to textual data. The general goal is, given one document, to efficiently identify the documents that are closest to the document under consideration, in terms of cosine similarity computed on the tf- idf representations of the documents. As a test case, we will use the relatively large Amazon Fine Food Reviews dataset, available at https://www.kaggle.com/datasets/ snap/amazon-fine-food-reviews and already used in a previous notebook. Please refer to the Colab notebook on exploratory analysis of the above dataset for dataset
retrieval and text feature extraction. In more detail you should:

1) Implement all necessary text prepropressing required to transform each review into a tf-idf vector. To this purpose, you can reuse (and improve!) the code available in the above mentioned notebook on exploratory analysis of the Amazon Fine Food Reviews dataset.

*Here I have performed a preprocessing, from the original review to the final proprocessed text, in which my aim has been to reduce unuseful words in order to identify the topic and the contents of the text.
The frist idea is to put it in lower case,  then to, as seen in class, rebuild all the contraction that leads to misunderstanding.
Going forward to eliminate from the text the unusefull punctuation and stopwords.
After it to obtain a set of token starting from sentences I've applied a tokenizer and then finally for each word in order to have just the root so the meaning, but not the ending (unusefull), I've applied a lemmatizer.
Putting all these word again togheter we have a new preprocessed text.
On this one is easy to apply the TFID tranformation that, put into a multidimentional space each document, basically respecting a distance-topic criteria.*




In [ ]:
Text=Text.apply(lambda text:"".join([char.lower() for char in text ]))
Text=Text.apply(lambda text:multiple_replace(contractions_dict, text))
Text=Text.apply(lambda text:"".join([char for char in text if char not in string.punctuation]))
Text= Text.apply(lambda text: remove_stopwords(text))
Text= Text.apply(lambda text: word_tokenize(text))
Text= Text.apply(lambda text: [ ps.stem(text[i]) for i in range(len(text))])
Text=Text.apply(lambda text:" ".join([e for e in text]))
vectorizer =TfidfVectorizer()
TF_IDF= Z = vectorizer.fit_transform(Text) #essentially now wa are arrived in a 200000 dimentions space peppresentation for each document.
scipy.sparse.save_npz('/content/drive/MyDrive/TFIDF.npz',TF_IDF)

2) Implement a class that, given a collection of tf-idf vectors (each representing a review), returns the collection of signatures of the vectors, where each signature is computed by applying the hyperplane method as seen in class. The length of the signature should be m = r × b, where as usual r and b are the number of rows and bands (see next point) of the signature matrix and are input parameters.

*Given a so high dimention of the Tf-IDf I've performed using the hyperplane method, so once obtained a dimentionality reduction to m, equal to rows times bands, is possible to perform an LSH on the signatures matrix. 
The method of hyperplane consist in the comparison of each point with this plane and basically assign 0 or 1 if it is below or above.
So in this way we obtain a strong reduction.*


In [ ]:
class Hyperplane_method:
    def __init__(self,r,b):
      self.r=r
      self.b=b
    def signatures_maker(self,Z):
      HPPL=np.random.randn(Z.shape[1],self.r*self.b)
      R = Z@HPPL
      S = np.where(R>0,1,0)#now we are in a r*b dimentions space, so we have reduced.
      return S,HPPL

3) Implement a LSH class implementing the banding technique seen in class (note that the number of dictionaries to use is equal to b).

*Once obtained for each document a binary vector of size "m", we split this m into "b" bands with "r" numbers in each one.
After have computed a transformation of all the numbers in a band, and if 2 docs have the same value for a band I'll consider them similar. And using a set I've obtained directly the pairs of corrispondence.*

In [ ]:
class LSH:
  def __init__(self,r=200,b=20):
    self.r=r
    self.b=b
  def buckets_compiler(self,S):
    list_dictionaries={}
    for band in range(b):
      list_dictionaries[band]={}
    binary_column = 2**np.arange(self.r).reshape(-1,1)
    S = np.split(S, self.b, axis=1)
    S = np.hstack([M@binary_column for M in S])
    for d,s in enumerate(S):
      for i,k in enumerate(s):
        diz=list_dictionaries[i]
        if k in diz.keys():
          diz[k].append(d)
        else:
          diz[k]=[d]
    return list_dictionaries #Dictionary of dictionary to be open with band id and bucket id.
  def bucket_id(self,te,HPPL): #given a te(document of test set) return all the buckets id (keys) in which te is insert.
        R = te@HPPL 
        S = np.where(R>0,1,0) 
        binary_column = 2**np.arange(self.r).reshape(-1,1)
        S = np.split(S, self.b, axis=1)
        S = np.hstack([M@binary_column for M in S]) #array of b (number of bands) keys that refere to specific buckets.
        return S

## Testing. 
Remove a small (and uniform) sample of the reviews from the dataset (say, 0.5%, which corresponds to more or less 2500 reviews). This will be your test set, while the rest of the dataset will be your training set. For each test review: i) compute the exact subset A of reviews in the training set, whose cosine similarity with the review under consideration is above θ; ii) compute the same subset, for the same threshold, this time time approximately using LSH. In general, this will be a subset B that is different from A (of course, we would like B to be as close to A as possible); iii) compute Jacc(A,B), as a measure of how A is close to B; iv) keep track of the times needed to compute A using exact method and B using LSH. You should report averages (over the number of test reviews) of the values computed in ii) - iv).
Note that exact similarity computation is already available in python libraries.

*I've computed effectivly for each Tf-IDf vector in the test set, which are really duplicates for it, and then using the keys of each document to look into the buckets and  find all the similar documents.*

In [ ]:
Z=scipy.sparse.load_npz('/content/drive/MyDrive/TFIDF.npz')
train,test = train_test_split(Z, test_size=0.005)

r,b=30,30

H=Hyperplane_method(r,b)
S,HPPL=H.signatures_maker(train) 

lsh=LSH(r,b)
DIZ=lsh.buckets_compiler(S) 

In [ ]:
def compute_A(train,te,r,b):
  thresh=(1/b)**(1/r)
  amicizie=set()
  SIM=sklearn.metrics.pairwise.cosine_similarity(te,train)>thresh
  SIM=np.where(SIM)[1]
  if len(SIM)!=0:
    amicizie.update(SIM)
  return amicizie 

def compute_B(diz,te):
  keys=lsh.bucket_id(te,HPPL)[0]
  amicizie=set()
  for i in range(b):
    diz=DIZ[i]
    k=keys[i]
    if k in diz.keys():
          amicinuovi=set(diz[k])
          amicizie.update(amicinuovi)
  return amicizie

def Jacc_similarity(A,B):
  intersection=len(A.intersection(B))
  union=len(A.union(B))
  if union!=0:
    return intersection/union
  else:
    return 0

In [ ]:
all_jac_sim=[]
time_A=[]
time_B=[]
for te in tqdm(test,total=test.shape[0]):
  start = timeit.default_timer()
  A=compute_A(train,te,r,b)
  stop = timeit.default_timer()
  time_A.append(stop-start)
  start = timeit.default_timer()
  B=compute_B(DIZ,te)
  stop = timeit.default_timer()
  time_B.append(stop-start)
  J=Jacc_similarity(A,B)
  all_jac_sim.append(J)
  df = pd.DataFrame(list(zip(time_A,time_B,all_jac_sim)),columns =['Time A', 'Time B','Jaccard Similarity'])
print("the AVG Jaccad Similarity is: "+ str(sum(all_jac_sim)/len(all_jac_sim)))
print("the AVG Time A is: "+ str(sum(time_A)/len(time_A)))
print("the AVG Time B is: "+ str(sum(time_B)/len(time_B)))

  0%|          | 0/1421 [00:00<?, ?it/s]

  0%|          | 0/1422 [00:00<?, ?it/s]

the AVG Jaccad Similarity is: 0.3985574188787066
the AVG Time A is: 0.8055463025311304
the AVG Time B is: 0.0005109336908184206


*The result is enough to deem that LSH provide a good approximations (the jaccard symilarity among sets is quite high (~40%).
And also, once insert the train into the buckets (quite time consuming), the computation of LSH pairs is incredibly faster.
The average time for compute pairs with the cosine similarity on Tf_IDf vector is 0.8s instead with LSH buckets (just computing the hash keys) is 0.0005s (let's gooooo!).*

## Fixing the threshold.

Fixing the threshold θ is necessary to identify near neighbours and to fix the values of r and b. To this purpose, you need to get a feeling of the average pairwise cosine similarity between documents. A rudimentary way to proceed is the following (but you can think of something more principled of course): i) sample a small fraction of the reviews (e.g., 1% of them) uniformly at random and compute the average pairwise cosine similarity between them (note that this is likely to be a relatively small value); ii) fix the threshold θ to a value that is a few times larger than average pairwise cosine similarity, e.g., 5 times or more. Note that if the threshold is too high you might have very few (or no) near neighbours for most documents, while you might have too many if it is too low.

In [ ]:
T,Sample=train_test_split(Z, test_size=0.05)
cosine_similarity=sklearn.metrics.pairwise.cosine_similarity(te,train)#cosine similarity in the sample among docs
desiderata_thresh=np.mean(cosine_similarity)*7
print(str(round((desiderata_thresh)*100,2))+"%")

11.46%


*In order to obtain a better resut I might provide a threshold of 11% of similarity using row and bands such that (1/b)^(1/r)=0.11; of course taking into account a combination of the trade off.
Using a lot of rows an no band leads us to false negative, and doing it wih the contrary leads to self positive.*